In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import gc
from math import radians,asin
from sklearn.metrics import mean_squared_error,explained_variance_score,mean_absolute_error
from sklearn.model_selection import KFold,StratifiedKFold
import lightgbm as lgb
from tqdm import tqdm_notebook as tqdm
import numpy as np
import math
import os

warnings.filterwarnings('ignore')

# 读取port并清洗

In [2]:
port=pd.read_csv('./input/port.csv')

# 去除重名港口
port=port.drop_duplicates('TRANS_NODE_NAME').reset_index(drop=True)
# port=port.drop_duplicates(['LONGITUDE','LATITUDE']).reset_index(drop=True) # 按照经纬度去重是为了让按照经纬度匹配港口名称唯一


port['TRANS_NODE_NAME']=port['TRANS_NODE_NAME'].astype(str).str.lower()

port_log_lat=port[['TRANS_NODE_NAME','LONGITUDE','LATITUDE']]

# 这些值是缺失值，需要去除
port_log_lat=port_log_lat.loc[~((port_log_lat.LONGITUDE==0.831116)&(port_log_lat.LATITUDE==-0.386592))]
port_log_lat=port_log_lat.loc[~((port_log_lat.LATITUDE==0.831116)&(port_log_lat.LONGITUDE==-0.386592))]

port_log_lat['lon1']=port_log_lat['LONGITUDE'].map(lambda x:np.radians(x))
port_log_lat['lat1']=port_log_lat['LATITUDE'].map(lambda x:np.radians(x))

#  def function

In [3]:
def make_trance_len_1_to_nan(df): # 把trance长度为1的的样本重置trance为nan
    df['trance_str_len']=df['TRANSPORT_TRACE'].map(lambda x:len(str(x).split('-')))
    df.loc[df['trance_str_len']==1,'TRANSPORT_TRACE']=np.nan
    df=df.drop(columns=['trance_str_len'])
    return df

In [4]:
def split_dest_ori_from_trance(df):
    df['TRANSPORT_TRACE']=df['TRANSPORT_TRACE'].str.lower()
    df['ori_port']=df['TRANSPORT_TRACE'].map(lambda x: str(x).split('-')[0])
    df['dest_port']=df['TRANSPORT_TRACE'].map(lambda x: str(x).split('-')[-1])
    df.loc[df.TRANSPORT_TRACE.isna(),'ori_port']=np.nan
    df.loc[df.TRANSPORT_TRACE.isna(),'dest_port']=np.nan
    
    df['trance_isna']=0
    df.loc[df.TRANSPORT_TRACE.isna(),'trance_isna']=1
    
    return df

In [5]:
# 补充经纬度
def creat_log_lat_use_last_sample(df,port_df,type_str): #用最后一条样本和最前面的一条样本将trance缺失的匹配最近的port
    df1=df.loc[df.TRANSPORT_TRACE.isna()]  # 缺失的那最后一条样本填充
    if type_str=='dest':
        gps_last_time=df1.groupby('loadingOrder').tail(1)[['loadingOrder','longitude','latitude']]
    elif type_str=='ori':
        gps_last_time=df1.groupby('loadingOrder').head(1)[['loadingOrder','longitude','latitude']]

    a=gps_last_time['loadingOrder']
    log_dest=gps_last_time['longitude']
    lat_dest=gps_last_time['latitude']
    log_dest_map={i:j for i,j in zip(a,log_dest)}
    lat_dest_map={i:j for i,j in zip(a,lat_dest)}

    df1['log_'+type_str]=df1['loadingOrder'].map(log_dest_map)
    df1['lat_'+type_str]=df1['loadingOrder'].map(lat_dest_map)
        
    df2=df.loc[df.TRANSPORT_TRACE.notna()] # 未缺失的直接拿port匹配,但是在port里面并不是包含gps里面所有的港口
    log_map={i:j for i,j in zip(port_df['TRANS_NODE_NAME'],port_df['LONGITUDE'])}
    lat_map={i:j for i,j in zip(port_df['TRANS_NODE_NAME'],port_df['LATITUDE'])}
    if type_str=='dest':
        df2['log_dest']=df2['dest_port'].map(log_map)
        df2['lat_dest']=df2['dest_port'].map(lat_map)
    elif type_str=='ori':
        df2['log_ori']=df2['ori_port'].map(log_map)
        df2['lat_ori']=df2['ori_port'].map(lat_map)
    
    df1=df1.append(df2)
    df1=df1.sort_values(['loadingOrder','timestamp'])
    df1=df1.reset_index(drop=True)
    return df1
# 这个函数已经能够补全所有的出发和目的地的经纬度，但是TRANSPORT_TRACE为nan的目的地和出发地的经纬度需要更新

In [6]:
# 将trance缺省的值利用出发地与目的地的经纬度和port中的位置计算，匹配距离最近的port
# 同时生成目的地，出发地港口的特征
def find_dest_ori_port_name(df,port_log_lat):
    df_trance_nan=df.loc[df.TRANSPORT_TRACE.isna()]
    df_trance_nan=df_trance_nan.drop_duplicates(['loadingOrder']) # 一个样本对应一条记录
    df_trance_nan=df_trance_nan.reset_index(drop=True)

    df_trance_nan['lon2_dest']=df_trance_nan['log_dest'].map(lambda x:np.radians(x))
    df_trance_nan['lat2_dest']=df_trance_nan['lat_dest'].map(lambda x:np.radians(x))
    df_trance_nan['lon2_ori']=df_trance_nan['log_ori'].map(lambda x:np.radians(x))
    df_trance_nan['lat2_ori']=df_trance_nan['lat_ori'].map(lambda x:np.radians(x))

    def find_dest(x,df,type_str):
        if type_str=='ori':
            df['lon2']=x['lon2_ori']
            df['lat2']=x['lat2_ori']
        else:
            df['lon2']=x['lon2_dest']
            df['lat2']=x['lat2_dest']
        dlon=df['lon1']-df['lon2']
        dlat=df['lat1']-df['lat2']
        a=np.sin(dlat/2)**2+np.cos(df['lat1'])*np.cos(df['lat2'])*np.sin(dlon/2)**2
        dist=2*np.arcsin(np.sqrt(a))*6371 # 这里单位是千米
        df['dist']=dist

        port_name=df.loc[df['dist']==df['dist'].min()]['TRANS_NODE_NAME'].values
        return port_name[0]

    df_trance_nan['ori_port']=df_trance_nan[['lon2_ori','lat2_ori']].apply(find_dest,args=(port_log_lat,'ori',),axis=1)
    df_trance_nan['dest_port']=df_trance_nan[['lon2_dest','lat2_dest']].apply(find_dest,args=(port_log_lat,'dest',),axis=1)


    ori_name_map={i:j for i,j in zip(df_trance_nan['loadingOrder'],df_trance_nan['ori_port'])}
    dest_name_map={i:j for i,j in zip(df_trance_nan['loadingOrder'],df_trance_nan['dest_port'])}

    df.loc[df['TRANSPORT_TRACE'].isna(),'ori_port']=df['loadingOrder'].map(ori_name_map)
    df.loc[df['TRANSPORT_TRACE'].isna(),'dest_port']=df['loadingOrder'].map(dest_name_map)
    return df
# 这个函数将缺省值的港口名称匹配

In [7]:
def trance_nan_update_log_lat(df,port_df):
    log_map={i:j for i,j in zip(port_df['TRANS_NODE_NAME'],port_df['LONGITUDE'])}
    lat_map={i:j for i,j in zip(port_df['TRANS_NODE_NAME'],port_df['LATITUDE'])}
    
    
    df.loc[df.TRANSPORT_TRACE.isna(),'log_dest']=df['dest_port'].map(log_map)
    df.loc[df.TRANSPORT_TRACE.isna(),'lat_dest']=df['dest_port'].map(lat_map)
    
    df.loc[df.TRANSPORT_TRACE.isna(),'log_ori']=df['ori_port'].map(log_map)
    df.loc[df.TRANSPORT_TRACE.isna(),'lat_ori']=df['ori_port'].map(lat_map)
    
    return df

In [8]:
def calculate_last_gps_to_dest_dist(df,type_str):
    if type_str=='dest':
        gps_last_time=df.groupby('loadingOrder').tail(1)[['loadingOrder','longitude','latitude','log_dest','lat_dest','log_ori','lat_ori']]
    elif type_str=='ori':
        gps_last_time=df.groupby('loadingOrder').head(1)[['loadingOrder','longitude','latitude','log_dest','lat_dest','log_ori','lat_ori']]
    
    gps_last_time['lon1']=gps_last_time['log_'+type_str].map(lambda x:np.radians(x))
    gps_last_time['lon2']=gps_last_time['longitude'].map(lambda x:np.radians(x))
    gps_last_time['lat1']=gps_last_time['lat_'+type_str].map(lambda x:np.radians(x))
    gps_last_time['lat2']=gps_last_time['latitude'].map(lambda x:np.radians(x))
    dlon=gps_last_time['lon1']-gps_last_time['lon2']
    dlat=gps_last_time['lat1']-gps_last_time['lat2']
    a=np.sin(dlat/2)**2+np.cos(gps_last_time['lat1'])*np.cos(gps_last_time['lat2'])*np.sin(dlon/2)**2
    dist=2*np.arcsin(np.sqrt(a))*6371
    if type_str=='dest':
        gps_last_time['last_gps_to_dest_dist']=dist
        last_gps_to_dest_dist_map={i:j for i,j in zip(gps_last_time['loadingOrder'],gps_last_time['last_gps_to_dest_dist'])}
        df['last_gps_to_dest_dist']=df['loadingOrder'].map(last_gps_to_dest_dist_map)
    else:
        gps_last_time['latest_gps_to_ori_dist']=dist
        latest_gps_to_ori_dist_map={i:j for i,j in zip(gps_last_time['loadingOrder'],gps_last_time['latest_gps_to_ori_dist'])}
        df['latest_gps_to_ori_dist']=df['loadingOrder'].map(latest_gps_to_ori_dist_map)
    return df

In [9]:
def create_trance(df):
    df_na=df.loc[df.TRANSPORT_TRACE.isna()]

    df_na['TRANSPORT_TRACE']=df_na['ori_port']+'-'+df_na['dest_port']

    df.loc[df.TRANSPORT_TRACE.isna(),'TRANSPORT_TRACE']=df_na.TRANSPORT_TRACE
    return df

In [10]:
# def drop_some_sample(df):
#     df=df.loc[df.ori_port!=df.dest_port]
#     df=df.drop_duplicates(['loadingOrder','timestamp'])
#     df=df.loc[~df.loadingOrder.isin(['WS572098644244'])] # 这个单号0特别多，直接删除了
#     return df

In [11]:
def drop_some_sample(df):
#     df=df.loc[df.ori_port!=df.dest_port]
    df=df.drop_duplicates(['loadingOrder','timestamp'])
    df=df.loc[~df.loadingOrder.isin(['WS572098644244'])] # 这个单号0特别多，直接删除了
    # 循环删除每个运单头部全为零的数据
    num=0
    while(1):
        df_speed_head_0=df.groupby('loadingOrder').head(10)
        b=df_speed_head_0.groupby('loadingOrder')['speed'].mean().reset_index()
        b=b[b.speed==0]
        if len(b)<1:
            break
        else:
            df_speed_head_0=df.loc[df.loadingOrder.isin(b.loadingOrder)].groupby('loadingOrder').head(10)
            head_0_10_index=df_speed_head_0.loc[df_speed_head_0.speed==0].index
            df=df.loc[~df.index.isin(head_0_10_index)]
            num+=1
    print("迭代清除头部为0的次数：",num)
    
    # 循环删除每个运单尾部全为零的数据
    num=0
    while(1):
        df_speed_head_0=df.groupby('loadingOrder').tail(10)
        b=df_speed_head_0.groupby('loadingOrder')['speed'].mean().reset_index()
        b=b[b.speed==0]
        if len(b)<1:
            break
        else:
            df_speed_head_0=df.loc[df.loadingOrder.isin(b.loadingOrder)].groupby('loadingOrder').tail(10)
            head_0_10_index=df_speed_head_0.loc[df_speed_head_0.speed==0].index
            df=df.loc[~df.index.isin(head_0_10_index)]
            num+=1
    print("迭代清除尾部为0的次数：",num)
    # 下面还可以在接一个这样的循环，head(2),精修一下
    return df

In [12]:
def create_label(df):
    df['timestamp']=pd.to_datetime(df['timestamp'],infer_datetime_format=True,utc=True)
    df_max=df.groupby(['loadingOrder'])['timestamp'].agg(['max']).reset_index()
    df_max.columns=['loadingOrder','arrival_time']
#     df_min_max['label']=(df_min_max['onboardDate']-df_min_max['timestamp']).dt.total_seconds()//3600
    
    arrival_time_map={i:j for i,j in zip(df_max['loadingOrder'],df_max['arrival_time'])}
    df['arrival_time']=df['loadingOrder'].map(arrival_time_map)
    del arrival_time_map
    
    df_min=df.groupby(['loadingOrder'])['timestamp'].agg(['min']).reset_index()
    df_min.columns=['loadingOrder','onboardDate']
    onboard_time_map={i:j for i,j in zip(df_min['loadingOrder'],df_min['onboardDate'])}
    df['onboardDate']=df['loadingOrder'].map(onboard_time_map)
    del onboard_time_map
    
    df['label']=(df['arrival_time']-df['timestamp']).dt.total_seconds()//3600
    gc.collect()
    
    return df
# gps.drop(columns=['latest_gps_to_ori_dist'],inplace=True)
# gps=create_label(gps)

In [13]:
def cal_speed(df,shift_num):
    df['lat_shift']=df.groupby('loadingOrder')['latitude'].shift(shift_num)
    df['log_shift']=df.groupby('loadingOrder')['longitude'].shift(shift_num)
    df.loc[df['lat_shift'].isna(),'lat_shift']=df.loc[df['lat_shift'].isna()]['latitude']
    df.loc[df['log_shift'].isna(),'log_shift']=df.loc[df['log_shift'].isna()]['longitude']
    
    df['log_shift']=df['log_shift'].map(lambda x:np.radians(x))
    df['lon2']=df['longitude'].map(lambda x:np.radians(x))
    df['lat_shift']=df['lat_shift'].map(lambda x:np.radians(x))
    df['lat2']=df['latitude'].map(lambda x:np.radians(x))
    
    dlon=df['log_shift']-df['lon2']
    dlat=df['lat_shift']-df['lat2']
    
    a=np.sin(dlat/2)**2+np.cos(df['lat_shift'])*np.cos(df['lat2'])*np.sin(dlon/2)**2
    dist=2*np.arcsin(np.sqrt(a))*6371

    df['diff_dist']=dist
    df=df.drop(columns=['lat_shift','log_shift','lon2','lat2'])
    
    df['time_diff']=df.groupby('loadingOrder')['timestamp'].diff(shift_num).dt.total_seconds()/3600
    if shift_num<0:
        df['time_diff']=np.abs(df['time_diff'])
    df.loc[df['time_diff'].isna(),'time_diff']=np.inf
    
    df['speed_'+str(shift_num)]=df['diff_dist']/df['time_diff']
    
    df.loc[df['time_diff']==0,'speed_'+str(shift_num)]=0
    df.loc[df['speed_'+str(shift_num)]<2,'speed_'+str(shift_num)]=0
    
    df.loc[df['speed_'+str(shift_num)]>80,'speed_'+str(shift_num)]=df.loc[df['speed_'+str(shift_num)]>80]['speed']
    
    df=df.drop(columns=['speed'])
    
    df=df.rename(columns={'speed_'+str(shift_num):'speed'})
    
    df=df.drop(columns=['time_diff','diff_dist'])
    return df

In [14]:
def find_first_time_to_dest_dist_about_100(df):
    
    def calculate_curr_to_dest_dist(df): 
        df['lon1']=df['log_dest'].map(lambda x:np.radians(x))
        df['lon2']=df['longitude'].map(lambda x:np.radians(x))
        df['lat1']=df['lat_dest'].map(lambda x:np.radians(x))
        df['lat2']=df['latitude'].map(lambda x:np.radians(x))
        dlon=df['lon1']-df['lon2']
        dlat=df['lat1']-df['lat2']
        a=np.sin(dlat/2)**2+np.cos(df['lat1'])*np.cos(df['lat2'])*np.sin(dlon/2)**2
        dist=2*np.arcsin(np.sqrt(a))*6371
        df['curr_to_dest_length']=dist
        drop_cols=['lon1','lon2','lat1','lat2']
        df.drop(columns=drop_cols,inplace=True)
        return df
    df=calculate_curr_to_dest_dist(df)
    
    df['curr_to_dest_length']=df['curr_to_dest_length'].astype(int)
    
#     df['hist_speed_mean']=df.groupby('loadingOrder')['speed'].transform('mean')
    
    df['curr_dist_minus_100']=np.abs(df['curr_to_dest_length']-50)
    
    a=df[df['curr_dist_minus_100']<50] # 只取距离港口100千米范围内的数据
    
    a=a.sort_values(['loadingOrder','curr_dist_minus_100','timestamp'])
#     print(a[['loadingOrder','curr_dist_minus_100','timestamp','curr_to_dest_length']])
    
    a=a.groupby('loadingOrder').head(5)[['loadingOrder','timestamp','curr_dist_minus_100']] # 选取候选的top5
    
    a=a.sort_values(['loadingOrder','timestamp'])
    a=a.groupby('loadingOrder').head(1)[['loadingOrder','timestamp','curr_dist_minus_100']] # 在候选top5里面选取时间最早的
    
    last_time_map={i:j for i,j in zip(a['loadingOrder'],a['timestamp'])}
    
    df['last_time']=df['loadingOrder'].map(last_time_map)
    
    df=df.loc[df['timestamp']<=df['last_time']]
    
    df=df.drop(columns=['curr_dist_minus_100','curr_to_dest_length','last_time'])
        
    return df

# 清洗GPS

In [13]:
# 将原始数据排序
%%time
gps=pd.read_csv('./input/gps.csv',header=None)

gps.columns=['loadingOrder','carrierName','timestamp','longitude',
                  'latitude','vesselMMSI','speed','direction','vesselNextport',
                  'vesselNextportETA','vesselStatus','vesselDatasource','TRANSPORT_TRACE']

gps=gps.sort_values(['loadingOrder','timestamp'])
gps.to_csv('./input/gps.csv',index=False)

CPU times: user 26min 46s, sys: 2min 21s, total: 29min 7s
Wall time: 28min 23s


In [4]:
# 生成chunk，并且优化，这样可以在分块读取的时候可以多读几个chunk
if not os.path.exists('./input/GPS'):
    os.makedirs('./input/GPS')
for index,chunk in tqdm(enumerate(pd.read_csv('./input/gps.csv',chunksize=10000000))):
    chunk.columns=['loadingOrder','carrierName','timestamp','longitude',
                  'latitude','vesselMMSI','speed','direction','vesselNextport',
                  'vesselNextportETA','vesselStatus','vesselDatasource','TRANSPORT_TRACE']
    chunk['longitude']=chunk['longitude'].astype('float32')
    chunk['latitude']=chunk['latitude'].astype('float32')
    chunk['direction']=chunk['direction'].astype('int32')
#     chunk=find_dest_ori_port(chunk) 
    save_path='./input/GPS/'+str(index)+'_chunk.pkl'
    chunk=chunk.reset_index(drop=True)
    chunk.to_pickle(save_path)
    gc.collect()

In [15]:
%%time
# 读取的chunk必须是已经排好序的数据，即必须相对源数据排好序，然后使用源数据生成chunk
if not os.path.exists('./input/GPS_clean_for_chunk'):
    os.makedirs('./input/GPS_clean_for_chunk')

last_id=[] # python的list .append是没有返回值的，而pandas.append是由返回值的
chuck_path='./input/GPS/'
use_cols=['loadingOrder','carrierName','timestamp','longitude',
                  'latitude','vesselMMSI','speed','direction','TRANSPORT_TRACE']

id_num=0
test=pd.read_csv('./input/test_A.csv')
test['TRANSPORT_TRACE']=test['TRANSPORT_TRACE'].str.lower()
test_trance=test.TRANSPORT_TRACE.unique().tolist()
print(test_trance)
trance_notna_has_in_gps=[]
trance_na_has_in_gps=[]

for index in tqdm(range(16)):  # 16G内存不能存放，占时先处理|大约1亿的数据并保存
    print(f'正在处理第 {index+1} 个chunk')
    file_name=os.path.join(chuck_path,str(index)+'_chunk'+'.pkl')
    print(file_name)
    chunk=pd.read_pickle(file_name)
    chunk=chunk[use_cols]
    chunk=chunk.drop_duplicates(['loadingOrder','timestamp','latitude','longitude','vesselMMSI','carrierName'])
    chunk=make_trance_len_1_to_nan(chunk) # 把len(trance)==1的重置为nan
    chunk=split_dest_ori_from_trance(chunk) # 分离出目的地和出发地
    chunk=creat_log_lat_use_last_sample(chunk,port_log_lat,'dest') # 建立目的地和出发地的经纬度
    chunk=creat_log_lat_use_last_sample(chunk,port_log_lat,'ori')
    chunk=chunk.loc[(chunk.log_dest.notna())&(chunk.log_ori.notna())]
    chunk=find_dest_ori_port_name(chunk,port_log_lat) # 对trance为缺省值的匹配港口名称
    chunk=trance_nan_update_log_lat(chunk,port_log_lat) # 根据港口名称，更新trance为缺省的目的地出发地的经纬度
    chunk=find_first_time_to_dest_dist_about_100(chunk) # 把一些塞港，到港又开走的数据截断，这样所有数据最后一条数据大约都在距离港口50千米
    chunk=calculate_last_gps_to_dest_dist(chunk,'dest') # 计算最后一个gps和第一个gps距离目的港口和出发港口的距离
    chunk=calculate_last_gps_to_dest_dist(chunk,'ori')
    chunk=create_trance(chunk) # 为trance为缺省的填充
    id_num+=chunk.loadingOrder.nunique()
    print(f'当前运单号个数：{id_num}')
    last_id.append(chunk.tail(1)['loadingOrder'].values.tolist()[0])
    
    chunk_trance_notna=chunk[chunk.trance_isna==0].TRANSPORT_TRACE.unique().tolist()
    chunk_trance_na=chunk[chunk.trance_isna==1].TRANSPORT_TRACE.unique().tolist()
    for trance in test_trance:
        if trance in chunk_trance_notna and trance not in trance_notna_has_in_gps:
            trance_notna_has_in_gps.append(trance) # 记录原本就在gps里trance非空的trance在test中出现的路径，应该有17个
        if trance in chunk_trance_na and trance not in trance_na_has_in_gps:
            trance_na_has_in_gps.append(trance)  # 记录trance填充后在测试集中出现的trance的个数
    print("test中的trance出现在gps里trance非空数据中trance出现的类别数",len(trance_notna_has_in_gps),trance_notna_has_in_gps)
    print("test中的trance出现在gps里trance为空数据中trance出现的类别数",len(trance_na_has_in_gps),trance_na_has_in_gps)
    chunk_name=os.path.join('./input/GPS_clean_for_chunk/','chunk_'+str(index)+'.pkl')
    chunk.to_pickle(chunk_name)
    del chunk
    gc.collect()
# gps=gps.reset_index(drop=True)
# gps=gps.loc[~gps.loadingOrder.isin(last_id)] # 将每个chunk的每个末尾的数据去掉，因为可能会因为截断，导致label误差很大
# gps.to_csv('./input/gps_chunk.csv',index=False) # 把全部数据保存好之后在选满足条件的样本

['cnytn-nzakl', 'cnytn-mxzlo', 'cnhkg-arbue', 'cnytn-matng', 'hkhkg-frfos', 'cnnsa-nawvb', 'hungary-hon', 'cnshk-clvap', 'cnytn-paonx', 'cnshk-zadur', 'cnshk-esalg', 'cnnsa-ghtem', 'ltklj-deham-hkhkg', 'cnytn-rtm', 'cnshk-mxzlo', 'cnytn-pacol', 'cobun-hkhkg', 'cnshk-sikop', 'cnshk-pkqct', 'cnytn-arena', 'cnshk-mytpp', 'cnshk-sgsin', 'cnytn-twkhh', 'cnsha-pamit', 'cnshk-lbbey', 'cnhkg-mxzlo', 'cnshk-inmun', 'cnshk-grpir', 'cnytn-cavan', 'cnytn-mytpp-egsue-matng-macas', 'cnytn-matng-sndkr', 'cnshk-hkhkg-cnnbg-cnsha-mxzlo-mxlzc-gtprq', 'cnytn-cnxam-cnsgh-paonx-pamit-gtiz4-hnpcr-pamit-paonx-crlio', 'cnytn-twkhh-cnsha-cnnbg-mxzlo-svaqj', 'cnshk-bdcgp', 'cnytn-hkhkg-sgsin-brrio-brssz-brpng-brnvt-uymvd', 'cnytn-mytpp-frfos-frmrs-egsue-tntun', 'sikop-hkhkg', 'cnshk-hkhkg-twkhh-cnnbg-cnsha-cntao-krpus-mxzlo-pablb-pamit-coctg-jmkin-dohai-anphi-gpptp-ttpos', 'cnnsa-sgsin-mytpp-nawvb-cgpnr-ngapp', 'cnshk-brssz', 'cnytn-brssz', 'cnshk-grpir-itspe-itgoa', 'brssz-hkhkg', 'cnshk-hkhkg-twkhh-cnnbg-cnsh

正在处理第 1 个chunk
./input/GPS/0_chunk.pkl
当前运单号个数：1071
test中的trance出现在gps里trance非空数据中trance出现的类别数 57 ['cnytn-nzakl', 'cnytn-mxzlo', 'cnytn-matng', 'cnshk-clvap', 'cnytn-paonx', 'cnshk-zadur', 'cnnsa-ghtem', 'cnytn-rtm', 'cnshk-mxzlo', 'cnshk-sikop', 'cnshk-pkqct', 'cnytn-arena', 'cnshk-sgsin', 'cnsha-pamit', 'cnytn-cavan', 'cnytn-mytpp-egsue-matng-macas', 'cnshk-hkhkg-cnnbg-cnsha-mxzlo-mxlzc-gtprq', 'cnytn-twkhh-cnsha-cnnbg-mxzlo-svaqj', 'cnshk-bdcgp', 'cnytn-hkhkg-sgsin-brrio-brssz-brpng-brnvt-uymvd', 'sikop-hkhkg', 'cnnsa-sgsin-mytpp-nawvb-cgpnr-ngapp', 'cnshk-brssz', 'cnytn-brssz', 'cnshk-grpir-itspe-itgoa', 'brssz-hkhkg', 'cnytn-sgsin-mtmla-dzalg', 'cnshk-esval', 'cnytn-mytpp-matng-nlrtm-deham', 'cnshk-sgsin-mtmla-sikop', 'cnnsa-sgsin-mytpp-tglfw-ghtem', 'cnhkg-gtprq', 'sikop-cnytn', 'cnshk-sgsin-mypkg-inmun-pkkhi-pkbqm', 'cnnsa-mytpp-sgsin-zacpt-cgpnr-galbv-cmkbi', 'cnytn-gbsou', 'cnshk-sgsin-aejea-qahmd-sadmm-omsoh', 'cnshk-mytpp-muplu-zadur', 'cnytn-bdcgp', 'cnshk-bhbah', 'cnnsa-sg

正在处理第 6 个chunk
./input/GPS/5_chunk.pkl
当前运单号个数：6908
test中的trance出现在gps里trance非空数据中trance出现的类别数 75 ['cnytn-nzakl', 'cnytn-mxzlo', 'cnytn-matng', 'cnshk-clvap', 'cnytn-paonx', 'cnshk-zadur', 'cnnsa-ghtem', 'cnytn-rtm', 'cnshk-mxzlo', 'cnshk-sikop', 'cnshk-pkqct', 'cnytn-arena', 'cnshk-sgsin', 'cnsha-pamit', 'cnytn-cavan', 'cnytn-mytpp-egsue-matng-macas', 'cnshk-hkhkg-cnnbg-cnsha-mxzlo-mxlzc-gtprq', 'cnytn-twkhh-cnsha-cnnbg-mxzlo-svaqj', 'cnshk-bdcgp', 'cnytn-hkhkg-sgsin-brrio-brssz-brpng-brnvt-uymvd', 'sikop-hkhkg', 'cnnsa-sgsin-mytpp-nawvb-cgpnr-ngapp', 'cnshk-brssz', 'cnytn-brssz', 'cnshk-grpir-itspe-itgoa', 'brssz-hkhkg', 'cnytn-sgsin-mtmla-dzalg', 'cnshk-esval', 'cnytn-mytpp-matng-nlrtm-deham', 'cnshk-sgsin-mtmla-sikop', 'cnnsa-sgsin-mytpp-tglfw-ghtem', 'cnhkg-gtprq', 'sikop-cnytn', 'cnshk-sgsin-mypkg-inmun-pkkhi-pkbqm', 'cnnsa-mytpp-sgsin-zacpt-cgpnr-galbv-cmkbi', 'cnytn-gbsou', 'cnshk-sgsin-aejea-qahmd-sadmm-omsoh', 'cnshk-mytpp-muplu-zadur', 'cnytn-bdcgp', 'cnshk-bhbah', 'cnnsa-sg

正在处理第 10 个chunk
./input/GPS/9_chunk.pkl
当前运单号个数：11663
test中的trance出现在gps里trance非空数据中trance出现的类别数 79 ['cnytn-nzakl', 'cnytn-mxzlo', 'cnytn-matng', 'cnshk-clvap', 'cnytn-paonx', 'cnshk-zadur', 'cnnsa-ghtem', 'cnytn-rtm', 'cnshk-mxzlo', 'cnshk-sikop', 'cnshk-pkqct', 'cnytn-arena', 'cnshk-sgsin', 'cnsha-pamit', 'cnytn-cavan', 'cnytn-mytpp-egsue-matng-macas', 'cnshk-hkhkg-cnnbg-cnsha-mxzlo-mxlzc-gtprq', 'cnytn-twkhh-cnsha-cnnbg-mxzlo-svaqj', 'cnshk-bdcgp', 'cnytn-hkhkg-sgsin-brrio-brssz-brpng-brnvt-uymvd', 'sikop-hkhkg', 'cnnsa-sgsin-mytpp-nawvb-cgpnr-ngapp', 'cnshk-brssz', 'cnytn-brssz', 'cnshk-grpir-itspe-itgoa', 'brssz-hkhkg', 'cnytn-sgsin-mtmla-dzalg', 'cnshk-esval', 'cnytn-mytpp-matng-nlrtm-deham', 'cnshk-sgsin-mtmla-sikop', 'cnnsa-sgsin-mytpp-tglfw-ghtem', 'cnhkg-gtprq', 'sikop-cnytn', 'cnshk-sgsin-mypkg-inmun-pkkhi-pkbqm', 'cnnsa-mytpp-sgsin-zacpt-cgpnr-galbv-cmkbi', 'cnytn-gbsou', 'cnshk-sgsin-aejea-qahmd-sadmm-omsoh', 'cnshk-mytpp-muplu-zadur', 'cnytn-bdcgp', 'cnshk-bhbah', 'cnnsa-

正在处理第 14 个chunk
./input/GPS/13_chunk.pkl
当前运单号个数：16585
test中的trance出现在gps里trance非空数据中trance出现的类别数 82 ['cnytn-nzakl', 'cnytn-mxzlo', 'cnytn-matng', 'cnshk-clvap', 'cnytn-paonx', 'cnshk-zadur', 'cnnsa-ghtem', 'cnytn-rtm', 'cnshk-mxzlo', 'cnshk-sikop', 'cnshk-pkqct', 'cnytn-arena', 'cnshk-sgsin', 'cnsha-pamit', 'cnytn-cavan', 'cnytn-mytpp-egsue-matng-macas', 'cnshk-hkhkg-cnnbg-cnsha-mxzlo-mxlzc-gtprq', 'cnytn-twkhh-cnsha-cnnbg-mxzlo-svaqj', 'cnshk-bdcgp', 'cnytn-hkhkg-sgsin-brrio-brssz-brpng-brnvt-uymvd', 'sikop-hkhkg', 'cnnsa-sgsin-mytpp-nawvb-cgpnr-ngapp', 'cnshk-brssz', 'cnytn-brssz', 'cnshk-grpir-itspe-itgoa', 'brssz-hkhkg', 'cnytn-sgsin-mtmla-dzalg', 'cnshk-esval', 'cnytn-mytpp-matng-nlrtm-deham', 'cnshk-sgsin-mtmla-sikop', 'cnnsa-sgsin-mytpp-tglfw-ghtem', 'cnhkg-gtprq', 'sikop-cnytn', 'cnshk-sgsin-mypkg-inmun-pkkhi-pkbqm', 'cnnsa-mytpp-sgsin-zacpt-cgpnr-galbv-cmkbi', 'cnytn-gbsou', 'cnshk-sgsin-aejea-qahmd-sadmm-omsoh', 'cnshk-mytpp-muplu-zadur', 'cnytn-bdcgp', 'cnshk-bhbah', 'cnnsa

In [17]:
# 如何去除那种进港又出来的数据，截断离港口最近的GPS，后面的去除

In [16]:
%%time

if not os.path.exists('./input/GPS_clean_for_sample'):
    os.makedirs('./input/GPS_clean_for_sample')


chunk_path='./input/GPS_clean_for_chunk/'
for index in range(16):
    print(f'正在处理第{index+1}个chunk')
    file_name=os.path.join(chunk_path,'chunk_'+str(index)+'.pkl')
    chunk=pd.read_pickle(file_name)
    print(f'订单个数{chunk.loadingOrder.nunique()}')

#     chunk=chunk.loc[chunk.last_gps_to_dest_dist<20000]
    chunk['timestamp']=pd.to_datetime(chunk['timestamp'],infer_datetime_format=True,utc=True)
        
    chunk=cal_speed(chunk,1)
    chunk=drop_some_sample(chunk)
    chunk=create_label(chunk)
    save_name='./input/GPS_clean_for_sample/'+'chunk_'+str(index)+'.pkl'
    chunk.to_pickle(save_name)

正在处理第1个chunk
订单个数1071
迭代清除头部为0的次数： 37
迭代清除尾部为0的次数： 5
正在处理第2个chunk
订单个数1077
迭代清除头部为0的次数： 30
迭代清除尾部为0的次数： 10
正在处理第3个chunk
订单个数1210
迭代清除头部为0的次数： 32
迭代清除尾部为0的次数： 9
正在处理第4个chunk
订单个数1097
迭代清除头部为0的次数： 77
迭代清除尾部为0的次数： 48
正在处理第5个chunk
订单个数1223
迭代清除头部为0的次数： 46
迭代清除尾部为0的次数： 5
正在处理第6个chunk
订单个数1230
迭代清除头部为0的次数： 53
迭代清除尾部为0的次数： 9
正在处理第7个chunk
订单个数1146
迭代清除头部为0的次数： 37
迭代清除尾部为0的次数： 10
正在处理第8个chunk
订单个数1148
迭代清除头部为0的次数： 49
迭代清除尾部为0的次数： 9
正在处理第9个chunk
订单个数1213
迭代清除头部为0的次数： 38
迭代清除尾部为0的次数： 22
正在处理第10个chunk
订单个数1248
迭代清除头部为0的次数： 52
迭代清除尾部为0的次数： 10
正在处理第11个chunk
订单个数1190
迭代清除头部为0的次数： 38
迭代清除尾部为0的次数： 10
正在处理第12个chunk
订单个数1185
迭代清除头部为0的次数： 51
迭代清除尾部为0的次数： 325
正在处理第13个chunk
订单个数1261
迭代清除头部为0的次数： 53
迭代清除尾部为0的次数： 5
正在处理第14个chunk
订单个数1286
迭代清除头部为0的次数： 34
迭代清除尾部为0的次数： 8
正在处理第15个chunk
订单个数1247
迭代清除头部为0的次数： 35
迭代清除尾部为0的次数： 4
正在处理第16个chunk
订单个数178
迭代清除头部为0的次数： 12
迭代清除尾部为0的次数： 1
CPU times: user 1h 34min 30s, sys: 1h 11min 13s, total: 2h 45min 43s
Wall time: 51min 51s


In [17]:
%%time
def down_sample_by_time(df):
    df['timestamp']=pd.to_datetime(df['timestamp'],infer_datetime_format=True,utc=True)
    id_unique=df['loadingOrder'].unique().tolist()
    print(f'当前df订单号个数为{len(id_unique)}')
    df_s=df.sample(0)
    
    df['time_str']=df['timestamp']
    df['time_str']=df['time_str'].map(lambda x:x.strftime("%Y-%m-%d"))
    df['hour']=df['timestamp'].dt.hour
    df['id_time']=df['loadingOrder']+df['timestamp'].astype(str)
    
    df_group=df.groupby(['loadingOrder','time_str','hour'])['timestamp'].min().reset_index()
    df_group['id_time']=df_group['loadingOrder']+df_group['timestamp'].astype(str)
    
    df=df.loc[df.id_time.isin(df_group['id_time'])]
    
    df.drop_duplicates('id_time',inplace=True)
    df.drop(columns=['id_time','time_str','hour'],inplace=True)
    return df
            


cols=['loadingOrder', 'carrierName', 'timestamp', 'longitude', 'latitude',
       'vesselMMSI', 'speed', 'direction', 'TRANSPORT_TRACE', 'ori_port',
       'dest_port', 'trance_isna', 'log_dest', 'lat_dest', 'log_ori',
       'lat_ori', 'last_gps_to_dest_dist', 'latest_gps_to_ori_dist',
       'arrival_time', 'onboardDate', 'label']
gps=pd.DataFrame(columns=cols)
chunk_path='./input/GPS_clean_for_sample/'
for index in tqdm(range(16)):
    print(f'正在处理第{index+1}个chunk')
    file_name=os.path.join(chunk_path,'chunk_'+str(index)+'.pkl')
    chunk=pd.read_pickle(file_name)
    print("原chunk中订单号个数：",chunk.loadingOrder.nunique())
    chunk=down_sample_by_time(chunk)
    gps=gps.append(chunk)
gps=gps.sort_values(['loadingOrder','timestamp'])
gps=gps.reset_index(drop=True)
gps.to_pickle('./input/gps_sample-8-10_BDC.pkl')

正在处理第1个chunk
原chunk中订单号个数： 1067
当前df订单号个数为1067
正在处理第2个chunk
原chunk中订单号个数： 1073
当前df订单号个数为1073
正在处理第3个chunk
原chunk中订单号个数： 1204
当前df订单号个数为1204
正在处理第4个chunk
原chunk中订单号个数： 1095
当前df订单号个数为1095
正在处理第5个chunk
原chunk中订单号个数： 1217
当前df订单号个数为1217
正在处理第6个chunk
原chunk中订单号个数： 1222
当前df订单号个数为1222
正在处理第7个chunk
原chunk中订单号个数： 1142
当前df订单号个数为1142
正在处理第8个chunk
原chunk中订单号个数： 1144
当前df订单号个数为1144
正在处理第9个chunk
原chunk中订单号个数： 1211
当前df订单号个数为1211
正在处理第10个chunk
原chunk中订单号个数： 1242
当前df订单号个数为1242
正在处理第11个chunk
原chunk中订单号个数： 1186
当前df订单号个数为1186
正在处理第12个chunk
原chunk中订单号个数： 1181
当前df订单号个数为1181
正在处理第13个chunk
原chunk中订单号个数： 1256
当前df订单号个数为1256
正在处理第14个chunk
原chunk中订单号个数： 1279
当前df订单号个数为1279
正在处理第15个chunk
原chunk中订单号个数： 1242
当前df订单号个数为1242
正在处理第16个chunk
原chunk中订单号个数： 178
当前df订单号个数为178

CPU times: user 25min 3s, sys: 9min 40s, total: 34min 44s
Wall time: 19min 33s


# 第二次清洗GPS

In [26]:
gps=pd.read_pickle('./input/gps_sample-8-10_BDC.pkl')

In [29]:
gps=gps[gps.last_gps_to_dest_dist<100]

gps['arrival_time_num']=gps.groupby('loadingOrder')['arrival_time'].transform('nunique')
gps=gps[gps.arrival_time_num==1]
gps=gps.drop(columns=['arrival_time_num'])

gps=gps.drop_duplicates(['TRANSPORT_TRACE','timestamp','longitude','latitude','vesselMMSI','carrierName','label'])

gps=gps.drop(columns=['trance_isna','last_gps_to_dest_dist', 'latest_gps_to_ori_dist','arrival_time'])

gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('cnytn','cnshk')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('shekou','cnshk')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('hkhkg','hkg')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('cnhkg','hkg')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('hongkong','hkg')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('ytn','hkg')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('ltklj-hk2_sf','hkg')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('hkgg kong_china','hkg')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('mtmal','mtmar')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('bu','buenosaires')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('hk2_sf','hkg')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('hk1_macaud','hkg')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('hk1_sino','hkg')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('hon','hkg')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('arbuenosairese','buenosaires')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('cobuenosairesn','buenosaires')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('buenosairesenosaires','buenosaires')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('cnshk','hkg')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('cnnsa','hkg')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('cvpra','cvrai')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('manzanillo_panama','paonx')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('panama','paonx')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('pacol','paonx')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('crpli','crlio')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('portoftanjungpelepas_malaysia','mytpp')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('nzakl','akl')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('sgsin','sin')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('cnsgh','shanghai')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('csh1_shhlw','shanghai')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('brsts','brssz')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('tza','bzbze')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('twkao','twkhh')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('cnkhs','twkhh')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('cndcw','hkg')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('cndcb','hkg')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('cndcb','cncwn')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('mylpk','mypkg')
gps['TRANSPORT_TRACE']=gps['TRANSPORT_TRACE'].str.replace('mywsp','mypkg')

gps['ori_port']=gps['TRANSPORT_TRACE'].map(lambda x: str(x).split('-')[0])
gps['dest_port']=gps['TRANSPORT_TRACE'].map(lambda x: str(x).split('-')[-1])

gps=gps.loc[~(gps['dest_port']==gps['ori_port'])]

In [4]:
gps.to_pickle('./input/gps_sample-8-10_BDC.pkl')

In [5]:
print(gps.loadingOrder.nunique(),len(gps))

9934 2613777
